Link to dataset: https://www.kaggle.com/datasets/jackksoncsie/spam-email-dataset 

In [1]:
import numpy as np 
import pandas as pd 
import os, csv, pathlib

  from pandas.core import (


Load the dataset.

In [2]:
raw_data_path = pathlib.Path("data\\raw_data.csv")
raw_data = pd.read_csv(raw_data_path)
raw_data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


Write a function that splits the dataset into training, validation and testing datasets.

In [3]:
from sklearn.model_selection import train_test_split

def split_data(df, randomState):
    train_data, temp_data = train_test_split(df, test_size = 0.3, random_state=randomState)
    val_data, test_data = train_test_split(temp_data, test_size = 0.5, random_state=randomState)
    return train_data, val_data, test_data

In [4]:
randomState = 2024
train_data_1, val_data_1, test_data_1 = split_data(raw_data, randomState)

Save the training, validation and testing datasets as csv files.

In [5]:
current_directory = pathlib.Path.cwd()

In [6]:
train_data_1.to_csv(current_directory / "data\\train.csv")

In [7]:
val_data_1.to_csv(current_directory / "data\\val.csv")

In [8]:
test_data_1.to_csv(current_directory / "data\\test.csv")